In [ ]:
#Import all the necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [ ]:
#Read the Dataset (Training + Test Set)
training_set = pd.read_csv('MNIST_ASL\\sign_mnist_train.csv')
test_set = pd.read_csv('MNIST_ASL\\sign_mnist_test.csv')

In [ ]:
#Separate the Pixels from the labels
x_train = training_set.iloc[:,1:].values  #Read all the rows from the 2nd column to the end
y_train = training_set.iloc[:,0].values    #Read all the rows only the 1st column
#mnist dataset has the label in the first column and the pixels in the remaining column
#size of reach row 28x28+1=784+1 [image'er gray scale value]
x_test = test_set.iloc[:,1:].values   #Read all the rows from the 2nd column to the end
y_test = test_set.iloc[:,0].values   #Read all the rows only the 1st column

In [ ]:
print(y_train.shape,y_test.shape)

(27455,) (7172,)


In [ ]:
#Reshape the Test and Training Data Sets into 28*28 Images
X=[]
Xt=[]
#Training Set Images
for i in x_train:
    X.append(i.reshape((28,28))) 
    '''
    #reshaping each row [1x784] of my training dataset into a 28x28 matrix
    '''

#Convert to Numpy Array
x_train = np.array(X)
y_train = np.array(y_train)#.reshape(-1)

#Test Set Images
for i in x_test:
    Xt.append(i.reshape((28,28)))

#Convert to Numpy Array
x_test = np.array(Xt)
y_test = np.array(y_test)#.reshape(-1)

In [ ]:
#Reshape for CNN
x_train = x_train.reshape(-1,28,28,1)
y_train = np.eye(26)[y_train]

x_test = x_test.reshape(-1,28,28,1)
y_test = np.eye(26)[y_test]

print(x_train.shape,y_train[0].shape)
print(x_test.shape, y_test[0].shape)


(27455, 28, 28, 1) (26,)
(7172, 28, 28, 1) (26,)


In [ ]:
#Normalize the Data
x_train = x_train/255.0
x_test = x_test/255.0


In [ ]:
print(y_train[0])

[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]


In [ ]:
#Make Your Model
model = tf.keras.models.Sequential([
    #1st Convolutional Layer
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(2,2),

    #2nd Convolutional Layer
    tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    #3rd Convolutional Layer
    tf.keras.layers.Conv2D(256, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    #Perform Regularization via Dropout
    tf.keras.layers.Dropout(0.5),

    #Flatten the Data
    tf.keras.layers.Flatten(),

    #Dense Layer-1
    tf.keras.layers.Dense(256, activation='relu'),

    #Dropout Layer
    tf.keras.layers.Dropout(0.5),

    #Last Dense Layer
    tf.keras.layers.Dense(26, activation='softmax')
])

In [ ]:
#Compile The Model
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics='accuracy') 

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 256)         295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 256)         0         
_________________________________________________________________
dropout (Dropout)            (None, 3, 3, 256)         0

In [ ]:
#Train The Model
hist = model.fit(x_train, y_train, epochs = 12, batch_size = 32, steps_per_epoch = x_train.shape[0]//32, validation_data = (x_test, y_test),
    validation_steps = x_test.shape[0]//32)

Epoch 1/12
857/857 [==============================] - 62s 72ms/step - loss: 2.2654 - accuracy: 0.3124 - val_loss: 0.2690 - val_accuracy: 0.9097
Epoch 2/12
857/857 [==============================] - 62s 72ms/step - loss: 0.2397 - accuracy: 0.9197 - val_loss: 0.1616 - val_accuracy: 0.9516
Epoch 3/12
857/857 [==============================] - 63s 74ms/step - loss: 0.1031 - accuracy: 0.9658 - val_loss: 0.1360 - val_accuracy: 0.9562
Epoch 4/12
857/857 [==============================] - 65s 76ms/step - loss: 0.0649 - accuracy: 0.9787 - val_loss: 0.1258 - val_accuracy: 0.9655
Epoch 5/12
857/857 [==============================] - 66s 77ms/step - loss: 0.0438 - accuracy: 0.9851 - val_loss: 0.1490 - val_accuracy: 0.9568
Epoch 6/12
857/857 [==============================] - 60s 71ms/step - loss: 0.0533 - accuracy: 0.9831 - val_loss: 0.1163 - val_accuracy: 0.9604
Epoch 7/12
857/857 [==============================] - 59s 69ms/step - loss: 0.0373 - accuracy: 0.9869 - val_loss: 0.0961 - val_accuracy:

In [ ]:
score = model.evaluate(x_test, y_test, verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

Test loss: 0.1385076493024826
Test accuracy: 0.9723926186561584


In [ ]:
# Saving trained model
model.save('model.h5')